<a href="https://colab.research.google.com/github/muhammad-usama-aleem/Object-Detection-using-tensorflow-api/blob/main/Finally%20training%20started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 69331, done.
remote: Total 69331 (delta 0), reused 0 (delta 0), pack-reused 69331
Receiving objects: 100% (69331/69331), 577.32 MiB | 27.60 MiB/s, done.
Resolving deltas: 100% (48851/48851), done.


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [1]:
import os

images_dir = "/content/gdrive/MyDrive/car_ims/car_ims/car_ims/"
path, dirs, files = next(os.walk("/content/gdrive/MyDrive/car_ims/car_ims/car_ims"))
file_count = len(files)
print(file_count)
print(path)

16185
/content/gdrive/MyDrive/car_ims/car_ims/car_ims


In [3]:
from scipy.io import loadmat
import cv2
import shutil
import pandas as pd
import os

resp = loadmat("/content/cars_annos.mat")

In [4]:
resp.keys()

dict_keys(['__header__', '__version__', '__globals__', 'annotations', 'class_names'])

In [5]:
resp.get("annotations")[0]

array([(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       (array(['car_ims/000002.jpg'], dtype='<U18'), array([[48]], dtype=uint8), array([[24]], dtype=uint8), array([[441]], dtype=uint16), array([[202]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       (array(['car_ims/000003.jpg'], dtype='<U18'), array([[7]], dtype=uint8), array([[4]], dtype=uint8), array([[277]], dtype=uint16), array([[180]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       ...,
       (array(['car_ims/016183.jpg'], dtype='<U18'), array([[25]], dtype=uint8), array([[32]], dtype=uint8), array([[587]], dtype=uint16), array([[359]], dtype=uint16), array([[196]], dtype=uint8), array([[1]], dtype=uint8)),
       (array(['car_ims/016184.jpg'], dtype='<U18'), array([[56]], dtype=uint8), array([[60]],

In [6]:
len(resp.get("annotations")[0])

16185

In [7]:
len(resp.get("class_names")[0])

196

In [8]:
classes = []
for class_name in resp.get("class_names")[0]:
    classes.append(class_name[0]) 

# **Create CSV File**

In [9]:
def get_data(test_type, directory):
    data = []
    if not os.path.exists(directory): os.mkdir(directory)
    for annot in resp.get("annotations")[0]:
        test = int(annot[6][0][0])
        if test == test_type: continue


        image_name = annot[0][0].split("/")[-1]
        print(image_name)
        image = cv2.imread(images_dir + image_name)
        shutil.copy(images_dir + image_name, f"{directory}/" + image_name)
        h, w, _ = image.shape
        xmin = int(annot[1][0][0])
        ymin = int(annot[2][0][0])
        xmax = int(annot[3][0][0])
        ymax = int(annot[4][0][0])
        class_name = classes[annot[5][0][0] - 1]

        data.append({
            'filename' : image_name, "width" : w, "height" : h, 
            'class' : class_name,
            'xmin' : xmin, 'ymin' : ymin, 'xmax' : xmax, 
            'ymax' : ymax})
        
    df = pd.DataFrame(data)
    return df

In [10]:
# !rm -rf /content/train

In [ ]:
# Get train data
df = get_data(0, "/content/train")
df.to_csv("train_labels.csv", index=None)

# Get train data
df = get_data(1, "/content/test")
df.to_csv("test_labels.csv", index=None)

In [12]:
import os

_, _, files = next(os.walk("/content/test"))
file_count = len(files)
print(file_count)

8144


In [13]:
with open("labels.pbtxt", "w") as file:
    for index, cls in enumerate(classes):
        row = "item {\n\tid: " + str(index + 1) + "\n\tname: '" + cls +"'\n}\n"
        file.write(row)

In [15]:
'''
paste this label dict in your '/content/models/research/preprocessing' generate_tfrecords.py
OR
make a labels.json file and use it in generate_tfrecords.py as we are going to do

'''

import json
labels = {cls:i + 1 for i, cls in enumerate(classes)}
with open('labels.json', "w") as file:
    json.dump(labels, file)
labels

{'AM General Hummer SUV 2000': 1,
 'Acura Integra Type R 2001': 6,
 'Acura RL Sedan 2012': 2,
 'Acura TL Sedan 2012': 3,
 'Acura TL Type-S 2008': 4,
 'Acura TSX Sedan 2012': 5,
 'Acura ZDX Hatchback 2012': 7,
 'Aston Martin V8 Vantage Convertible 2012': 8,
 'Aston Martin V8 Vantage Coupe 2012': 9,
 'Aston Martin Virage Convertible 2012': 10,
 'Aston Martin Virage Coupe 2012': 11,
 'Audi 100 Sedan 1994': 17,
 'Audi 100 Wagon 1994': 18,
 'Audi A5 Coupe 2012': 13,
 'Audi R8 Coupe 2012': 15,
 'Audi RS 4 Convertible 2008': 12,
 'Audi S4 Sedan 2007': 24,
 'Audi S4 Sedan 2012': 23,
 'Audi S5 Convertible 2012': 21,
 'Audi S5 Coupe 2012': 22,
 'Audi S6 Sedan 2011': 20,
 'Audi TT Hatchback 2011': 19,
 'Audi TT RS Coupe 2012': 25,
 'Audi TTS Coupe 2012': 14,
 'Audi V8 Sedan 1994': 16,
 'BMW 1 Series Convertible 2012': 27,
 'BMW 1 Series Coupe 2012': 28,
 'BMW 3 Series Sedan 2012': 29,
 'BMW 3 Series Wagon 2012': 30,
 'BMW 6 Series Convertible 2007': 31,
 'BMW ActiveHybrid 5 Sedan 2012': 26,
 'BMW

# Generating Tfrecord file

In [ ]:
# !rm -rf /content/models

In [16]:
import os

print(os.getcwd())

/content


In [17]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [18]:
import os

print(os.getcwd())

/content/models/research


In [19]:
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 8.6 MB/s 
     |████████████████████████████████| 352 kB 48.4 MB/s 
     |████████████████████████████████| 2.2 MB 34.4 MB/s 
     |████████████████████████████████| 23.4 MB 482 kB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 234 kB 47.4 MB/s 
     |████████████████████████████████| 47.7 MB 71 kB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 99 kB 8.2

In [20]:
print(os.getcwd())
# !python object_detection/builders/model_builder_tf1_test.py
!python object_detection/builders/model_builder_tf2_test.py

/content/models/research
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-26 07:36:58.863524: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0226 07:36:59.407399 140344161339264 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.88s
I0226 07:36:59.766451 140344161339264 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.88s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.72s
I0226 07:37:00.49

In [21]:
print(os.getcwd())

/content/models/research


In [ ]:
'''
if error is this one:

Traceback (most recent call last):
  File "generate_tfrecord.py", line 76, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/app.py", line 48, in run
    _sys.exit(main(_sys.argv[:1] + flags_passthrough))
  File "generate_tfrecord.py", line 69, in main
    tf_example = create_tf_example(row)
  File "generate_tfrecord.py", line 60, in create_tf_example
    'image/object/class/label': dataset_util.int64_list_feature(classes),
  File "/home/j/Lib/models/object_detection/utils/dataset_util.py", line 26, in int64_list_feature
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
TypeError: None has type NoneType, but expected one of: int, long


then you should do this:

"The solution is a simple one.

You just need to change the else statement so it returns 0 instead of "None".

def class_text_to_int(row_label): if row_label == '<YOUR LABEL NAME>': return 1 else: return 0

You can find def class_text_to_int in the generate_tfrecord.py file.


'''

In [23]:
%cd '/content/models/research/preprocessing'
print(os.getcwd())
!python generate_tfrecords.py --csv_input=/content/train_labels.csv --image_dir=/content/train --output_path=/content/train.record

/content/models/research/preprocessing
/content/models/research/preprocessing
Successfully created the TFRecords: /content/train.record


In [24]:
!python generate_tfrecords.py --csv_input=/content/test_labels.csv --image_dir=/content/test --output_path=/content/test.record

Successfully created the TFRecords: /content/test.record


In [25]:
# !rm -rf /content/train 
# !rm -rf /content/test

# **detection** 
1- make a directory in the object detection folder named as 'standford_train'(in our case). 

2- choose a model from 'models/research/object_detection/g3doc/tf2_detection_zoo.md', we are choosing 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8' then download it and copy 'pipeline.config' and paste it in standford_train directory 

3- change the num_classes to the our total number of classes, should change batch_size to less than 5, change 'fine_tune_checkpoint_type = classification' to 'fine_tune_checkpoint_type = detection'  

4- 'fine_tune_checkpint' file should be in the downloaded folder

In [26]:
%cd '/content/models/research/object_detection'

/content/models/research/object_detection


In [28]:
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52

Found existing installation: opencv-python-headless 4.5.5.62
Uninstalling opencv-python-headless-4.5.5.62:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.62.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-64ac49e1.so.58.91.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-4b79e479.so.58.45.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-805734e8.so.56.51.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-018b8c17.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dist

In [ ]:
'''
if training stuck because of memory, then either reduce the batch size or reduce the height and width of the image 
in "pipeline.cofig" 
'''

In [ ]:
!python model_main_tf2.py --model_dir=/content/models/research/object_detection/standford_train --pipeline_config_path=/content/models/research/object_detection/standford_train/pipeline.config --alsologtostderr

2022-02-26 08:34:06.826672: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0226 08:34:06.902426 139628655433600 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0226 08:34:06.907952 139628655433600 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0226 08:34:06.908156 139628655433600 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0226 08:34:07.093298 139628655433600 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [ ]:
print(os.getcwd())